## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

#### Описание данных:

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

In [1]:
import pandas as pd #импортирую pandas, читаю файл и распечатываю первые 10 строк для ознакомления
data = pd.read_csv('/datasets/data.csv')
print(data.shape)
data.head(10)


(21525, 12)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
# бросаются в глаза разные регистры, посмотрим информацию о таблице
print(data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


Видим, что всего 21525 строк, в столбцах days_employed и total_income есть **пропущенные значения** (в количестве 2174, т.е. 10%), причём кол-во пропущенных значений в обоих столбцах одинаковое. 
В столбце days_employed есть отрицательные значения. Скорее всего, это ошибка записи данных и дефис был распознан как минус. Добавим новый столбец years_employed, взяв модуль стажа и переведя его в годы, так удобнее проводить оценки.

In [3]:
data['years_employed'] = data['days_employed'].abs() / 365 
print(data.describe())
# 365 дней взято из-за того, что у многих клиентов и так разница между возрастом и стажем около 14 лет,
# если взять кол-во рабочих дней в году, то эта разница станет неразумной

           children  days_employed     dob_years  education_id  \
count  21525.000000   19351.000000  21525.000000  21525.000000   
mean       0.538908   63046.497661     43.293380      0.817236   
std        1.381587  140827.311974     12.574584      0.548138   
min       -1.000000  -18388.949901      0.000000      0.000000   
25%        0.000000   -2747.423625     33.000000      1.000000   
50%        0.000000   -1203.369529     42.000000      1.000000   
75%        1.000000    -291.095954     53.000000      1.000000   
max       20.000000  401755.400475     75.000000      4.000000   

       family_status_id          debt  total_income  years_employed  
count      21525.000000  21525.000000  1.935100e+04    19351.000000  
mean           0.972544      0.080883  1.674223e+05      183.328024  
std            1.420324      0.272661  1.029716e+05      380.906522  
min            0.000000      0.000000  2.066726e+04        0.066141  
25%            0.000000      0.000000  1.030532e+05    

Средний стаж нашей группы 183 года, хотя у 75% фигурантов стаж выглядит логично. Есть клиент(ы), которому 0 лет, вряд ли он успел взять кредит - это тоже **значения, требующие замены**. Значит, в данных есть артефакты и придётся делать замены на характерные величины. Посмотрим, сколько видов занятости в наших данных и какой в них разброс данных. 

In [4]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

У нас есть 8 типов занятости. 

In [5]:
data.groupby('income_type').median().sort_values(by = 'total_income', ascending = False)

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income,years_employed
income_type,,,,,,,,
предприниматель,0.0,-520.848083,42.5,0.0,0.5,0.0,499163.144947,1.426981
компаньон,0.0,-1547.382223,39.0,1.0,0.0,0.0,172357.950966,4.239403
госслужащий,0.0,-2689.368353,40.0,1.0,0.0,0.0,150447.935283,7.368132
сотрудник,0.0,-1574.202821,39.0,1.0,0.0,0.0,142594.396847,4.312884
безработный,0.5,366413.652744,38.0,0.5,0.5,0.5,131339.751676,1003.873021
пенсионер,0.0,365213.306266,60.0,1.0,0.0,0.0,118514.486412,1000.584401
студент,0.0,-578.751554,22.0,0.0,4.0,0.0,98201.625314,1.585621
в декрете,2.0,-3296.759962,39.0,1.0,0.0,1.0,53829.130729,9.032219


На первый взгляд, самый большой разброс данных, в колонке years_employed, посмотрим повнимательнее, построив сводную таблицу data_groupd_by_income_type.

In [6]:
data_groupd_by_income_type = data.groupby('income_type').agg({'years_employed': ['count', 'min', 'max', 'mean'], 'total_income': ['mean', 'median'], 'children': ['sum', 'mean'], 'dob_years': ['mean'], 'debt': ['sum']})
data_groupd_by_income_type

years_employed                                        \
                         count         min          max         mean   
income_type                                                            
безработный                  2  924.724567  1083.021476  1003.873021   
в декрете                    1    9.032219     9.032219     9.032219   
госслужащий               1312    0.109463    41.624746     9.314786   
компаньон                 4577    0.082727    48.261817     5.784998   
пенсионер                 3443  900.626632  1100.699727  1000.009565   
предприниматель              1    1.426981     1.426981     1.426981   
сотрудник                10014    0.066141    50.380685     6.373970   
студент                      1    1.585621     1.585621     1.585621   

                  total_income                children            dob_years  \
                          mean         median      sum      mean       mean   
income_type                                                                   
безработный      131339.751676  131339.751676        1  0.500000  38.000000   
в декрете         53829.130729   53829.130729        2  2.000000  39.000000   
госслужащий      170898.309923  150447.935283      908  0.622344  40.636737   
компаньон        202417.461462  172357.950966     3044  0.598623  39.697542   
пенсионер        137127.465690  118514.486412      509  0.132002  59.063019   
предприниматель  499163.144947  499163.144947        0  0.000000  42.500000   
сотрудник        161380.260488  142594.396847     7136  0.641784  39.821027   
студент           98201.625314   98201.625314        0  0.000000  22.000000   

                 debt  
                  sum  
income_type            
безработный         1  
в декрете           1  
госслужащий        86  
компаньон         376  
пенсионер         216  
предприниматель     0  
сотрудник        1061  
студент             0

В данных есть артефакты: 

- стаж всех безработных и пенсионеров неправильный, а количество пенсионеров не является пренебрежимо малой величиной, надо разбираться

Важные выводы для дальнейших изучений корелляций:

- дети распределены по типу занятости не одинаково
- практически все группы, кроме пенсионеров и студентов в среднем одинаковы по возрасту, можно не учитывать возрастной фактор.

In [7]:
data.sort_values(by = 'days_employed', ascending = False)[3440:3450]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
7229,1,328827.345667,32,среднее,1,гражданский брак,1,F,пенсионер,0,122162.965695,сыграть свадьбу,900.896837
14783,0,328795.726728,62,высшее,0,женат / замужем,0,F,пенсионер,0,79940.196752,на покупку своего автомобиля,900.810210
17782,0,328771.341387,56,среднее,1,женат / замужем,0,F,пенсионер,0,68648.047062,операции с коммерческой недвижимостью,900.743401
9328,2,328734.923996,41,высшее,0,женат / замужем,0,M,пенсионер,0,126997.497760,операции со своей недвижимостью,900.643627
20444,0,328728.720605,72,среднее,1,вдовец / вдова,2,F,пенсионер,0,96519.339647,покупка жилья для семьи,900.626632
17437,1,-24.141633,31,среднее,1,женат / замужем,0,F,сотрудник,1,166952.415427,высшее образование,0.066141
8336,0,-24.240695,32,высшее,0,Не женат / не замужем,4,M,сотрудник,0,124115.373655,получение дополнительного образования,0.066413
6157,2,-30.195337,47,среднее,1,гражданский брак,1,M,компаньон,0,231461.185606,свадьба,0.082727
9683,0,-33.520665,43,среднее,1,Не женат / не замужем,4,M,сотрудник,1,128555.897209,приобретение автомобиля,0.091837
2127,1,-34.701045,31,высшее,0,женат / замужем,0,F,компаньон,0,90557.994311,получение образования,0.095071


В этом срезе видно, что в данных сначала фигурировали отрицательные стажи, а затем положительные, но не адекватные по величине. Возможная причина в том, что "положительные" стажи указаны не в днях, а в часах. Если бы у нас стояла задача изучения возврата кредита в зависимости от стажа, эти артефакты стоило бы внимательно изучить. **Наше задание этого не подразумевает.** Тем не менее, в условиях задачи сказано, что артефанты нужно обработать, сделаем это после удаления дубликатов.

Наша задача - изучить зависимость факта возврата кредита от кол-ва детей и семейного положения. Посмотрим, какие в наших данных виды семейного положения и какой разброс по кол-ву детей.

In [8]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [9]:
data.groupby('family_status').agg({'debt': ['count', 'sum']})

debt     
                       count  sum
family_status                    
Не женат / не замужем   2813  274
в разводе               1195   85
вдовец / вдова           960   63
гражданский брак        4177  388
женат / замужем        12380  931

В наших данных пять видов семейного положения. Для детального рассмотрения распределения должников по семейному статусу нужно посчитать относительное кол-во должников в каждой группе.

In [10]:
data.groupby('children').agg({'debt': ['count', 'sum']})


debt      
          count   sum
children             
-1           47     1
 0        14149  1063
 1         4818   444
 2         2055   194
 3          330    27
 4           41     4
 5            9     0
 20          76     8

У 65% клиентов банка нет детей (14149/21525 = 0.65), в этой же группе самое больше количество должников, но в дальнейшем при анализе нужно смотреть относительное количество должников в каждой группе. 

### Вывод

Данные — статистика о платёжеспособности клиентов — представляют собой таблицу размером (21525, 12). В данных отсутствует параметр user ID, что потенциально может быть не удобно для анализа, особенно в поиске дубликатов. В данных есть артефакты, пропуски и потенциальные "нестыковки". Столбцы, которые нужно анализировать для ответа на поставленную задачу: 'children', 'family_status', 'family_status_id', 'debt'. Теоретически, возраст тоже может оказывать влияние, т.к. у более молодых людей дети маленькие, на них тратится бюджет и это может влиять на возврат кредита.
 
#### Предварительные выводы для дальнейшего анализа:
 
 - У 65% клиентов банка нет детей (14149/21525 = 0.65), в этой же группе самое больше количество должников, но в дальнейшем при анализе нужно смотреть относительное количество должников в каждой группе
 
 - В наших данных пять видов семейного положения. Для детального рассмотрения распределения должников по семейному статусу нужно посчитать относительное кол-во должников в каждой группе
 
#### Замечания:
 
 
 1. В столбцах days_employed и total_income есть пропущенные значения, причём кол-во пропущенных значений одинаковое, нужно разобраться, является ли это совпадение случайным, или нет
  
 1. В столбце children и days_employed есть отрицательные значения, которых быть не должно. Для нас важна правильность данных в столбце children
 
 1. Стаж безработных и пенсионеров содержит артефакты, но в нашу задачу не входит изучение возврата кредита в зависимости от стажа, тем не менее требуется обработать эти артефакты.
 
 1. Среди клиентов есть тот, чей возраст равен нулю, сомнительно, что у него есть кредитная история. 
 
 1. В столбце education использованы разные регистры, это может влиять на количество дубликатов.



### Шаг 2. Предобработка данных

### Обработка пропусков

Из выводов Шага1 ясно, что в столбцах days_employed и total_income есть пропущенные значения. Для начала посмотрим, как выглядят эти строчки.

In [11]:
print(data[data['days_employed'].isnull()].head())

    children  days_employed  dob_years education  education_id  \
12         0            NaN         65   среднее             1   
26         0            NaN         41   среднее             1   
29         0            NaN         63   среднее             1   
41         0            NaN         50   среднее             1   
55         0            NaN         54   среднее             1   

            family_status  family_status_id gender  income_type  debt  \
12       гражданский брак                 1      M    пенсионер     0   
26        женат / замужем                 0      M  госслужащий     0   
29  Не женат / не замужем                 4      F    пенсионер     0   
41        женат / замужем                 0      F  госслужащий     0   
55       гражданский брак                 1      F    пенсионер     1   

    total_income                           purpose  years_employed  
12           NaN                   сыграть свадьбу             NaN  
26           NaN          

Можно подумать, что в строках отсутствуют days_employed и total_income одновременно. Проверим эту гипотезу.

In [12]:
print(data.loc[(data['days_employed'].isnull()) & (data['total_income'].notna())])

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose, years_employed]
Index: []


В данных нет строк, где одно из значений в столбцах days_employed и total_income нулевое, а второе нет (достаточно было проверить одно условие, поскольку суммы нулевых значений в обоих столбцах равны). Пропуски количественных переменных заменим медианными значениями, поскольку они не фигурируют в дальнейшей аналитике.

In [13]:
days_employed_median = data['days_employed'].median()
years_employed_median = data['years_employed'].median()
total_income_median = data['total_income'].median()
data['days_employed'] = data['days_employed'].fillna(days_employed_median)
data['total_income'] = data['total_income'].fillna(total_income_median)
data['years_employed'] = data['years_employed'].fillna(years_employed_median)
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
years_employed      21525 non-null float64
dtypes: float64(3), int64(5), object(5)
memory usage: 2.1+ MB
None


Проверили, что в столбцах total_income и days_employed нет пропущенных значений. Теперь вспомним, что у нас есть клиенты с нулевым возастом. Посмотрим, сколько их и какие варианты параметра возраст встречаются в данных.

In [14]:
print(data['dob_years'].value_counts())

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64


Нулевой возраст клиента - это тоже пропущенное значение, но таких клиентов всего 101 человек. Можно заменить их возраст медианным значением. 

In [15]:
data[data['dob_years'] == 0].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль,949.429093
149,0,-2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем,7.299379
270,3,-1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью,5.130584
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости,1090.017986
1040,0,-1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль,3.172684


### Вывод

В данных были заменены значения 'NaN' в столбцах days_employed и total_income. Эти данные - количественные переменные, которые, как правило, заменяют на характерные значения. В данном случае они были заменены на медианные значения, поскольку дальше при анализе нам не нужно учитывать эти параметры, то нет смысла усложнять способ замены. 

В столбце возраст был 101 человек с нулевым возрастом (в данных всегда могут встречаться артефакты, например описки, когда человеку 30 и цифра 3 не была набрана), их мы тоже заменили на медианое значение, поскольку 101 человек - это 0.5% от суммарного количества людей в данных.

### Замена типа данных

В столбцах days_employed и total_income тип представленных данных float64, хотя по смыслу дни и годы не должны быть дробными, а доход может быть округлён до целого. Для такой замены используют метод astype:

In [17]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data['years_employed'] = data['years_employed'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
years_employed      21525 non-null int64
dtypes: int64(8), object(5)
memory usage: 2.1+ MB


### Вывод

Методом astype были заменены типы переменных float64 в столбцах days_employed, years_employed и total_income.

### Обработка дубликатов

Для начала посмотрим, сколько у нас полных дубликатов:

In [18]:
data[data.duplicated() == True]


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
2849,0,-1203,41,среднее,1,женат / замужем,0,F,сотрудник,0,145017,покупка жилья для семьи,6
4182,1,-1203,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,145017,свадьба,6
4851,0,-1203,60,среднее,1,гражданский брак,1,F,пенсионер,0,145017,свадьба,6
5557,0,-1203,58,среднее,1,гражданский брак,1,F,пенсионер,0,145017,сыграть свадьбу,6
7808,0,-1203,57,среднее,1,гражданский брак,1,F,пенсионер,0,145017,на проведение свадьбы,6
8583,0,-1203,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,145017,дополнительное образование,6
9238,2,-1203,34,среднее,1,женат / замужем,0,F,сотрудник,0,145017,покупка жилья для сдачи,6
9528,0,-1203,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,145017,операции со своей недвижимостью,6
9627,0,-1203,56,среднее,1,женат / замужем,0,F,пенсионер,0,145017,операции со своей недвижимостью,6
10462,0,-1203,62,среднее,1,женат / замужем,0,F,пенсионер,0,145017,покупка коммерческой недвижимости,6


Видим, что у всех дубликатов пропущены значения days_employed и total_income. Возможно, была неудачная попытка завести повторно данные, но уже со всеми параметрами.

In [19]:
data.duplicated().sum()

55

Дубликатов 55, но в выводах к шагу 1 упомянуто, что в столбце education использованы разные регистры. Из-за этого может оказаться, что на самом деле дубликатов больше. Проверим:

In [20]:
data['education'] = data['education'].str.lower()
data.duplicated().sum()

72

 Процентуально дубликатов меншье 1%, их можно удалить без потерь для последующего анализа:

In [21]:
data = data.drop_duplicates().reset_index(drop = True)
data.duplicated().sum()

0

После удаления дубликатов мы можем обработать артефакты: отрицательные значения в столбцах days_employed и children, завышенные значения стажа в столбце days_employed (когда значения станвятся положительными, они становятся неадекватно высокими). 

Все отрицательные значения в столбце children равны "-1". Скорее всего это ошибка при сборе данных, когда дефис был вопринят как минус. Значит, нужно заменить все значения на единицу:

In [22]:
data['children'] = data['children'].replace(-1, 1)

Теперь разберёмся с артефактами в days_employed. Из таблицы data_groupd_by_income_type мы знаем, что артефакты есть только у безработных и пенсионеров, причём безработных всего 2 человека, так что с ними можно поступить так же, как с пенсионерами. Вообще стоит отметить, что данные стажа вызывают подозрение: лица, которым в среднем 40 лет имеют стаж от 6 до 9 лет (что они вообще делали?!). Логично предположить, что пенсионерами становятся люди из всех остальных групп деятельности. Поэтому мы подсчитаем среднее значение разницы стажа и возраста (назовём эту разницу work_begin_mean), а затем заменим артефакты на возраст пенсионера минус посчитанная разница:

In [23]:
def work_begin_count(row):  
    if row['years_employed'] < 100:
        work_begin = row['dob_years'] - row['years_employed']
    if row['years_employed'] > 100:
        work_begin = 0
    return work_begin
data['work_begin'] = data.apply(work_begin_count, axis=1)
work_begin_mean = data['work_begin'].sum() / data[data['years_employed'] < 100]['children'].count()
work_begin_mean

34.48228565082186

Итак, в среднем разница между возрастом и стажем в нашей выборки 34 года. Заменим теперь артефакты:

In [24]:
def years_employed_change(row):
    if row['years_employed'] > 100:
        if row['dob_years'] - work_begin_mean >= 0:
            years_employed_right = row['dob_years'] - int(work_begin_mean)
        else:
            years_employed_right = -1 # если получатся отрицательные значения, то их нужно будет заменить
       
    if row['years_employed'] < 100:
        years_employed_right = row['years_employed']
    return years_employed_right   
data['years_employed_right'] = data.apply(years_employed_change, axis=1)
data['years_employed'] = data['years_employed_right']
del data['years_employed_right']

Проверим, что у нас не появились отрицательные стажи:

In [25]:
data[data['years_employed'] < 0]['children'].count()

16

Поскольку всего у 16 человек получились отрицательные значения, такое количество не вносит вклад в среднее значение в столбце. Произведём замену отрицательных величин на среднее значение в столбце:

In [26]:
data['years_employed'] = data['years_employed'].replace(-1, int(data['years_employed'].mean()))

Проверим, что произошла замена:

In [27]:
data[data['years_employed'] < 0]['children'].count()

0

### Вывод

Было обнаружено 72 строка-дубликат, у всех дубликатов пропущены значения days_employed и total_income. Возможно, была неудачная попытка завести повторно данные, но уже со всеми параметрами. Поскольку их число от общего количества данных - менее 1%, они были удалены без потерь для дальнейшей аналитики.

Произведена обработка артефактов в столбце years_employed: было подсчитано среднее значение разницы между возрастом и стажем клиента по данным, не содержащим артефакты. Затем артефакты были заменены на возраст минус подсчитанная разница. В результате образовалось 16 отрицательных значений стажа. Они были заменены на среднее значение стажа в данных. 

### Лемматизация

Посмотрим, какие есть цели кредита:

In [1]:
data['purpose'].unique()

NameError: name 'data' is not defined

Видим, что среди целей кредита есть одинаковые по смыслу, но записанные по-разному. Например, "на покупку автомобиля", "свой автомобиль" и т.д. Логично произвести лемматизацию целей кредита. В итоге все цели кредита можно свести к семи:

- ремонт
- строительство
- жильё
- автомобиль
- образование
- свадьба
- недвижимость

Важно искать в лемматизированных списках сначало слова ремонт и строительство, поскольку они могут быть вместе со словом жильё и недвижимость. Т.е. мы переделываем цель "строительство/ремонт жилья" в "строительство/ремонт", а не в "жильё".

Для лемматизации импортируем библиотеку pymystem3:

In [29]:
from pymystem3 import Mystem
m = Mystem()

Теперь преведём лемматизированные строки к краткому виду из одного слова:

In [30]:
def lemmas_clean(text):
    lemmas = m.lemmatize(text)
    try:
        if 'ремонт' in lemmas:
            return 'ремонт'
        elif 'строительство' in lemmas:
            return 'строительство'
        elif 'жилье' in lemmas:
            return 'жилье'
        elif 'автомобиль' in lemmas:
            return 'автомобиль'
        elif 'образование' in lemmas:
            return 'образование'
        elif 'свадьба' in lemmas:
            return 'свадьба'
        elif 'недвижимость' in lemmas:
            return 'недвижимость'
    except:
        return 'attention' 
    # возможно, какие-то цели не получится перевести в простой вид, нужно будет помотреть, где возникли ошибки

data['purpose_simple'] = data['purpose'].apply(lemmas_clean)
data[data['purpose_simple'] == 'attention']['children'].count() # посмотрим, возникли ли ошибки в процессе лемматизации

0

### Вывод

В исходных данных был столбец с целями кредита. Многие цели совпадали по смыслу, но различное их написание не давало возможности группировки. Поэтому была произведена лемматизация и упрощение лемм, т.е. такие цели как "покупка автомобиля" и "свой автомобиль" были переведены в цель "автомобиль". На данном этапе имеем семь различных целей: ремонт, строительство, жилье, автомобиль, образование, свадьба и недвижимость. 

Минусом такой обработких данных является взаимная зависимость групп "недвижимость" и "жильё". Для разделения этих двух групп требуется более тщательный анализ. Кроме того, такие цели как "покупка жилой недвижимости" имеют двоякую трактовку: речь может идти о покупки жилья для себя (где клиент будет жить сам), или недвижимости как актива.

### Категоризация данных

В предыдущем пункте уже практически была произведена категоризация данных по целям кредита с помощью лемматизации. Было отмечено, что группы "недвижимость" и "жильё" связаны. По сути к ним стоит также добавить и группу "строительство", поскольку речь идёт о сттроительстве недвижимости. Объединим эти три группы в одну - "недвижимость".

In [31]:
def purpose_category_maker(lemmas):
    if 'ремонт' in lemmas:
        return 'ремонт'
    elif 'строительство' in lemmas:
        return 'недвижимость'
    elif 'жилье' in lemmas:
        return 'недвижимость'
    elif 'автомобиль' in lemmas:
        return 'автомобиль'
    elif 'образование' in lemmas:
        return 'образование'
    elif 'свадьба' in lemmas:
        return 'свадьба'
    elif 'недвижимость' in lemmas:
        return 'недвижимость'
    
data['purpose_category'] = data['purpose_simple'].apply(purpose_category_maker)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,work_begin,purpose_simple,purpose_category
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23,19,жилье,недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11,25,автомобиль,автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15,18,жилье,недвижимость
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11,21,образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,19,0,свадьба,свадьба


Для создания сводной таблицы добавим к данным ещё один столбец "debt_category": если значение "debt" в данных единица, то этот клиент будет отнесён к категории "задолженность" и "нет задолженности" при нуле.

In [32]:
def debt_category_maker(debt):
    if debt == 0:
        return 'нет задолженности'
    if debt == 1:
        return 'задолженность'
    
data['debt_category'] = data['debt'].apply(debt_category_maker)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,work_begin,purpose_simple,purpose_category,debt_category
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23,19,жилье,недвижимость,нет задолженности
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11,25,автомобиль,автомобиль,нет задолженности
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15,18,жилье,недвижимость,нет задолженности
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11,21,образование,образование,нет задолженности
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,19,0,свадьба,свадьба,нет задолженности


Теперь можно делать сводную таблицу: группируем клиентов по категориям целей кредита и считаем, сколько среди них тех, у кого есть задолженность и тех, у кого нет. Разделим должников на общее количество людей в группе и получим долю должников (для наглядности в процентах):

In [33]:
data_groupd_by_purpose = data.pivot_table(
    index=['purpose_category', 'purpose_simple'], 
    columns='debt_category', values="debt",aggfunc='count')
data_groupd_by_purpose['процент_должников'] = round(data_groupd_by_purpose['задолженность'] / (data_groupd_by_purpose['задолженность'] + data_groupd_by_purpose['нет задолженности']) * 100)
                                                                                                         
data_groupd_by_purpose                                                                                        

debt_category                    задолженность  нет задолженности  \
purpose_category purpose_simple                                     
автомобиль       автомобиль                403               3903   
недвижимость     жилье                     273               3580   
                 недвижимость              330               4143   
                 строительство             144               1734   
образование      образование               370               3643   
ремонт           ремонт                     35                572   
свадьба          свадьба                   186               2137   

debt_category                    процент_должников  
purpose_category purpose_simple                     
автомобиль       автомобиль                    9.0  
недвижимость     жилье                         7.0  
                 недвижимость                  7.0  
                 строительство                 8.0  
образование      образование                   9.0  
ремонт           ремонт                        6.0  
свадьба          свадьба                       8.0

Далее в задаче нам нужно проанализировать, есть ли завсимость факта возврата кредита в срок от семейного статуса. Построим сводную таблицу зависимости наличия задолженности от семейного статуса data_groupd_by_family_status. Для этого сгруппируем данные по по параметру "family_status" и и подсчитаем, сколько всего клиентов с конкретным семейным статусом (столбец "count" считает и нули и единицы в столбце "debt" исходных данных, т.е. считает, сколько суммарно человек в этой категории) и сколько из них должников (в столбце "sum" додсчитаны суммы параметра "debt", т.е. сумма "единиц" = сумма должников). Для наглядности добавим столбец "процент_должников".

In [34]:
data_groupd_by_family_status = data.groupby('family_status').agg({'debt': ['count', 'sum']})
data_groupd_by_family_status['процент_должников'] = round(data_groupd_by_family_status['debt']['sum'] / data_groupd_by_family_status['debt']['count'] * 100)
data_groupd_by_family_status

debt      процент_должников
                       count  sum                  
family_status                                      
Не женат / не замужем   2810  274              10.0
в разводе               1195   85               7.0
вдовец / вдова           959   63               7.0
гражданский брак        4150  388               9.0
женат / замужем        12339  931               8.0

Далее в задаче нам нужно ответить на вопрос, есть ли зависимость факта возврата кредита в срок от наличия детей. Построим сводную таблицу зависимости количества от количества детей data_groupd_by_children. Принцип построения таблицы полностью аналогичен предыдущему.

In [35]:
data_groupd_by_children = data.groupby('children').agg({'debt': ['count', 'sum']})
data_groupd_by_children['процент_должников'] = round(data_groupd_by_children['debt']['sum'] / data_groupd_by_children['debt']['count'] * 100)
data_groupd_by_children


debt       процент_должников
          count   sum                  
children                               
0         14090  1063               8.0
1          4855   445               9.0
2          2052   194               9.0
3           330    27               8.0
4            41     4              10.0
5             9     0               0.0
20           76     8              11.0

Поскольку в задаче спрашивается зависимость от наличия детей, а не от их количества, мы можем разделить клиентов на тех, у кого есть дети, и на тех, у кого их нет. Для этого напишем функцию-категоризатор:

In [36]:
def children_category_maker(children):
    if children > 0:
        return 'есть дети'
    else:
        return 'нет детей'
    
data['children_category'] = data['children'].apply(children_category_maker)
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,work_begin,purpose_simple,purpose_category,debt_category,children_category
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23,19,жилье,недвижимость,нет задолженности,есть дети
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11,25,автомобиль,автомобиль,нет задолженности,есть дети
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15,18,жилье,недвижимость,нет задолженности,нет детей
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11,21,образование,образование,нет задолженности,есть дети
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,19,0,свадьба,свадьба,нет задолженности,нет детей
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21448,1,-4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,12,31,жилье,недвижимость,нет задолженности,есть дети
21449,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,33,0,автомобиль,автомобиль,нет задолженности,нет детей
21450,1,-2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,5,33,недвижимость,недвижимость,задолженность,есть дети
21451,3,-3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,8,30,автомобиль,автомобиль,задолженность,есть дети


In [37]:
data_groupd_by_children_existence = data.pivot_table(
    index=['children_category', 'children'], 
    columns='debt_category', values="debt",aggfunc='count')
data_groupd_by_children_existence

debt_category               задолженность  нет задолженности
children_category children                                  
есть дети         1                 445.0             4410.0
                  2                 194.0             1858.0
                  3                  27.0              303.0
                  4                   4.0               37.0
                  5                   NaN                9.0
                  20                  8.0               68.0
нет детей         0                1063.0            13027.0

У нас появился "NaN", но мы уже знаем, что у группы клиентов с пятью детьми нет задолженности. Заменим NaN на ноль и добавим столбец "процент_должников".

In [38]:
data_groupd_by_children_existence = data_groupd_by_children_existence.fillna(0)
data_groupd_by_children_existence['процент_должников'] = round(data_groupd_by_children_existence['задолженность'] / (data_groupd_by_children_existence['задолженность'] + data_groupd_by_children_existence['нет задолженности']) * 100)

data_groupd_by_children_existence


debt_category               задолженность  нет задолженности  \
children_category children                                     
есть дети         1                 445.0             4410.0   
                  2                 194.0             1858.0   
                  3                  27.0              303.0   
                  4                   4.0               37.0   
                  5                   0.0                9.0   
                  20                  8.0               68.0   
нет детей         0                1063.0            13027.0   

debt_category               процент_должников  
children_category children                     
есть дети         1                       9.0  
                  2                       9.0  
                  3                       8.0  
                  4                      10.0  
                  5                       0.0  
                  20                     11.0  
нет детей         0                       8.0

Более простой для анализа будет таблица, где подсчитан процент должников в каждой из двух групп "есть дети" и "нет детей":

In [39]:
data_groupd_by_children_existence2 = data.groupby('children_category').agg({'debt': ['count', 'sum']})
data_groupd_by_children_existence2['процент_должников'] = round(data_groupd_by_children_existence2['debt']['sum'] / data_groupd_by_children_existence2['debt']['count'] * 100)
data_groupd_by_children_existence2

debt       процент_должников
                   count   sum                  
children_category                               
есть дети           7363   678               9.0
нет детей          14090  1063               8.0

Кроме того, в задаче фигурирует вопрос о том, влияет ли уровень дохода клиентов на возврат кредита. Для ответа нужно произвести категоризацию данных по размеру дохода. В Шаге 1 была построена таблица по категориям вида деятельности клиентов data_groupd_by_income_type. Просмотрев диапазон доходов в каждой группе, представляется логичным поделить клиентов на три группы:
с доходами до 100000 (низкий доход), от 100000 до 200000 (средний доход) с высокими доходами - выше 200000. 

Напишем функцию-классификатор:

In [41]:
def income_category_maker(income):
    if income > 200000:
        return 'высокий_доход'
    if income > 100000:
        return 'средний_доход'
    else:
        return 'низкий_доход'
    
data['income_category'] = data['total_income'].apply(income_category_maker)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,work_begin,purpose_simple,purpose_category,debt_category,children_category,income_category
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23,19,жилье,недвижимость,нет задолженности,есть дети,высокий_доход
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11,25,автомобиль,автомобиль,нет задолженности,есть дети,средний_доход
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15,18,жилье,недвижимость,нет задолженности,нет детей,средний_доход
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11,21,образование,образование,нет задолженности,есть дети,высокий_доход
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,19,0,свадьба,свадьба,нет задолженности,нет детей,средний_доход


Теперь посмотрим, равномерно ли разделились клиенты на группы:

In [42]:
data['income_category'].value_counts()

средний_доход    11924
высокий_доход     5066
низкий_доход      4463
Name: income_category, dtype: int64

Клиенты поделены на три категории по доходам: средняя группа - самая многочисленная (что вполне логично), группа с низким доходом чуть меньше численности группы с высоким доходом. 

Нужно учитывать, что данное деление довольно условное. 
Теперь посторим сводную таблицу для изучения зависимости проблемы возврата кредита от дохода. Для этого сгруппируем данные по по параметру "income_category" и и подсчитаем, сколько всего клиентов с конкретным уровнем дохода (столбец "count" считает и нули и единицы в столбце "debt" исходных данных, т.е. считает, сколько суммарно человек в этой категории) и сколько из них должников (в столбце "sum" додсчитаны суммы параметра "debt", т.е. сумма "единиц" = сумма должников). Для наглядности добавим столбец "процент_должников".

In [43]:
data_groupd_by_income = data.groupby('income_category').agg({'debt': ['count', 'sum']})
data_groupd_by_income['процент_должников'] = round(data_groupd_by_income['debt']['sum'] / data_groupd_by_income['debt']['count'] * 100)

data_groupd_by_income

debt       процент_должников
                 count   sum                  
income_category                               
высокий_доход     5066   358               7.0
низкий_доход      4463   354               8.0
средний_доход    11924  1029               9.0

### Вывод

Была произведена категоризация данных по целям кредита с помощью лемматизации: выделено пять целей кредита и построена сводная таблица доли должников в зависимости от целей кредита data_groupd_by_purpose.

Для изучения зависимости возврата кредита в срок от количества детей, сначала была произведена категоризация клиентов по количеству детей. Но в формулировке задачи фигурирует именно наличие детей, а не их количество. Поэтому клиенты были разбиты на две категории: те, у кого есть дети и те, у кого их нет. Из выводов Шага1 следует, что у 65% клиентов нет детей, т.е. группы различны по численности. Также постоена сводная таблица доли должников в зависимости от наличия детей data_groupd_by_children_existence2.

Для ответа на вопрос о зависимости возврата кредита от семейного положения, данные сгруппированы в сводной таблице  по семейному положению
data_groupd_by_family_status.

Наиболее спорная категоризация сделана для изучения зависимости возврата кредита в срок от уровня дохода. Поскольку отсутсвуют указания от заказчика исследования о том, как категоризировать доход, было выделено три группы клиентов: с доходами до 100000 (низкий доход), от 100000 до 200000 (средний доход) с высокими доходами - выше 200000. По этим категориям были сгруппированы данные о возврате кредита в срок и построена сводная таблица data_groupd_by_income.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Посмотрим сводную таблицу data_groupd_by_children_existence ещё раз и переименуем в ней оси для простоты понимания:

In [44]:
data_groupd_by_children_existence2.set_axis(['численность_группы','число_должников','процент_должников'],axis='columns',inplace=True)
data_groupd_by_children_existence2

,численность_группы,число_должников,процент_должников
children_category,,,
есть дети,7363,678,9.0
нет детей,14090,1063,8.0


### Вывод

Как было указано в выводах к Шагу 1 у 65% клиентов нет детей. У 23% клиентов есть один ребёнок, а у 10% двое детей. Мы из таблицы data_groupd_by_children_existence видно, что доля клиентов-должников в категории "есть дети" выше, чем у бездетной категории. Речь идёт об отличии в один процент, что может быть объяснено погрешностью вычислений. Таким образом,  **нет зависимости между наличием детей и возвратом кредита в срок**.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Посмотрим таблицу data_groupd_by_family_status. Для наглядности переименуем столбцы "count" и "sum" на "численность группы" и "число должников" соответственно.

In [45]:
data_groupd_by_family_status.set_axis(['численность_группы','число_должников','процент_должников'],axis='columns',inplace=True)
data_groupd_by_family_status

,численность_группы,число_должников,процент_должников
family_status,,,
Не женат / не замужем,2810,274,10.0
в разводе,1195,85,7.0
вдовец / вдова,959,63,7.0
гражданский брак,4150,388,9.0
женат / замужем,12339,931,8.0


### Вывод

Из таблицы data_groupd_by_family_status видно, что **самая большая доля должников в группе "не женат/не замужем", самая меньшая доля должников в группе "вдовец/вдова"**. Различия между остальными группами лежат в пределах погрешности вычислений.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для ответа на этот вопрос посмотрим таблицу data_groupd_by_income. Для наглядности переименуем столбцы "count" и "sum" на "численность группы" и "число должников" соответственно.

In [46]:
data_groupd_by_income.set_axis(['численность_группы','число_должников','процент_должников'],axis='columns',inplace=True)
data_groupd_by_income


,численность_группы,число_должников,процент_должников
income_category,,,
высокий_доход,5066,358,7.0
низкий_доход,4463,354,8.0
средний_доход,11924,1029,9.0


### Вывод

Из таблицы data_groupd_by_income видно, что **самую большую сложность при возврате кредита испытывает группа клиентов со средним доходом по сравнению с клиентами с высоким доходом**. Но данный вывод сильно зависит от того, как производится классификация по группам дохода. Данный вопрос стоит согласовать с заказчиком исследования.

- Как разные цели кредита влияют на его возврат в срок?

Посмотрим на сводную таблицу data_groupd_by_purpose ещё раз:

In [47]:
data_groupd_by_purpose

debt_category                    задолженность  нет задолженности  \
purpose_category purpose_simple                                     
автомобиль       автомобиль                403               3903   
недвижимость     жилье                     273               3580   
                 недвижимость              330               4143   
                 строительство             144               1734   
образование      образование               370               3643   
ремонт           ремонт                     35                572   
свадьба          свадьба                   186               2137   

debt_category                    процент_должников  
purpose_category purpose_simple                     
автомобиль       автомобиль                    9.0  
недвижимость     жилье                         7.0  
                 недвижимость                  7.0  
                 строительство                 8.0  
образование      образование                   9.0  
ремонт           ремонт                        6.0  
свадьба          свадьба                       8.0

### Вывод

Мы видим, что расхождения в последнем стобце минимальны. **Меньше всего трудностей с возвратом кредитов испытывают те, кто берёт кредит на ремонт. При этом среди тех, кто берёт деньги на образование и на автомобиль, должников в 1,5 раза больше.**

### Шаг 4. Общий вывод

Целью исследования было изучение надёжности клиентов банка. А именно требовалось установить зависимость факта возврата кредита вовремя от ряда характеристик клиентов: их уровня дохода, семейного положения, наличия детей и целей кредита. 

Поскольку в задаче не поставлены условия категоризации клиентов по уровню дохода, было выделено три группы клиентов: с доходами до 100000 (низкий доход), от 100000 до 200000 (средний доход) с высокими доходами - выше 200000. Самую большую сложность при возврате кредита испытывает группа клиентов со средним доходом по сравнению с клиентами с высоким доходом. Различия составляют 2% и очевидным образом зависят от того, как производится классификация. Данный вопрос рекоммендуется согласовать с заказчиком исследования.

В ходе исследования было установлено, что клиенты по-разному возвращают кредит в зависимости от семейного положения: самая большая доля должников в группе "не женат/не замужем", самая меньшая доля должников в группе "вдовец/вдова". Речь идёт о незначительном различии в 2%. Промежуточные результаты демонстрируют группы "в разводе", "гражданский брак" и "женат / замужем". Разница их показателей находится в пределах погрешности рассчётов.

Данная выборка не демонстрирует зависимости между наличием детей и возвратом кредита в срок. Стоит отметить, что у 65% клиентов в наших данных не имеют детей. Возможно, результат исследования был бы более репрезентативным, если бы было больше данных о клиентах с разным количеством детей (отличным от нуля).

Цели кредита влияют на возврат кредита в срок. Так, меньше всего трудностей с возвратом кредитов испытывают клиенты, планирующие ремонт. При этом среди тех, кто берёт деньги на образование, или на автомобиль, должников в 1,5 раза больше чем в первой группе. Клиенты, берущие деньги на свадьбу, образование, или на операции с недвижимостью примерно одинаково ведут себя при возврате кредита.

Таким образом, были выявлены зависимости (или факт их отсутствия) возврата кредита вовремя от всех требуемых характеристик.